<a href="https://colab.research.google.com/github/Khadiza13/Bug-identify_attachment/blob/main/QA_and_Paraphrase_bug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import requests
import pandas as pd

In [ ]:
def qa_infer(question, context):
    url = "https://corpus.bangla.gov.bd/downstream-api/api/v1/QA/qa_infer"
    headers = {"Content-Type": "application/json"}

    payload = {"context": context, "question": question}

    response = requests.post(url, headers=headers, json=payload, verify=False)
    response.raise_for_status()  # Check for HTTP request errors
    result = response.json()
    answer = result['predictions']['answer']['text']

    return answer


In [ ]:
context = "ব্র্যাক ব্যাংক লিমিটেডে অভিজ্ঞতা ছাড়াই নিয়োগ বিজ্ঞপ্তি প্রকাশ করেছে । অফিসার পদে চাকরির সুযোগ দেবে প্রতিষ্ঠানটি । উক্ত পদে নারী - পুরুষ উভয়েই আবেদন করতে পারবেন । আগ্রহ ও যোগ্যতা থাকলে আপনিও আবেদন করতে পারেন । সম্পূর্ণ বিজ্ঞপ্তি বিস্তারিত দেওয়া হল :"
question = "কোথায় নিয়োগ বিজ্ঞপ্তি প্রকাশ হয়েছে?"

In [ ]:
qa_infer(question, context)

'ব্র্যাক ব্যাংক লিমিটেডে'

![BERT](https://production-media.paperswithcode.com/methods/new_BERT_Overall.jpg)

1. if out of context
2. multiple spans
3. big context
4. majority non-bangla text

In [ ]:
from datasets import load_dataset
dataset = load_dataset("csebuetnlp/squad_bn", trust_remote_code=True, split="test")



README.md:   0%|          | 0.00/9.17k [00:00<?, ?B/s]

squad_bn.py:   0%|          | 0.00/3.94k [00:00<?, ?B/s]

squad_bn.tar.bz2:   0%|          | 0.00/8.43M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
from tqdm.auto import tqdm

correct = 0
total = 0

anomalies = []
num_samples = 2500

subset = dataset.select(range(num_samples))

for sample in tqdm(subset):
    context = sample['context']
    question = sample['question']
    try:
        gt_ans = sample['answers']['text'][0]
    except IndexError:
        gt_ans = ""

    try:
        pred_ans = qa_infer(question, context)
        if gt_ans == pred_ans:
            correct += 1
        total += 1
    except requests.exceptions.HTTPError:
        anomalies.append({
            "context": context,
            "question": question,
            "gt_ans": gt_ans,
        })

print(f"Accuracy: {correct/total}")
df = pd.DataFrame.from_records(anomalies)
df.to_csv("anomalies.csv", index=False)

  0%|          | 0/2500 [00:00<?, ?it/s]

Accuracy: 0.1508169250104734


In [ ]:
df.head()

,context,question,gt_ans
0,রোনালদো (Portuguese pronunciation:[χoˈnawdu lu...,রোনাল্দো লুইস নাজারিও ডি লিমা কোন সালে ব্রাজিল...,২০১১
1,"with Sanskrit text, four English translations ...",হিন্দু পুরাণে মোট কতজন দেব দেবীর উল্ল্যেখ আছে ?,
2,চারুলতা-পরবর্তী বছরগুলোতে সত্যজিৎ ছিলেন বৈচিত্...,করণ জোহরের প্রযোজনা সংস্থাটির নাম কী ?,
3,"Burkert, Walter, (1977) 1985. Greek Religion, ...",গ্রিক পুরাণ মতে জিউসের স্ত্রী কে ছিলেন ?,
4,—Qualitative and quantitative scenarios and mo...,বিশ্বায়নের উৎপত্তির ইতিহাস নিয়ে ভারতে সর্বপ্র...,


In [ ]:
# https://corpus.bangla.gov.bd/api/user
# https://corpus.bangla.gov.bd/downstream-api/api/v1/PARAPHRASE/paraphrase
def paraphrase(source_text, target_text):
    url = "https://corpus.bangla.gov.bd/downstream-api/api/v1/PARAPHRASE/paraphrase?source_text=%E0%A6%A4%E0%A7%81%E0%A6%AE%E0%A6%BF+%E0%A6%95%E0%A6%BF+%E0%A6%86%E0%A6%9C%E0%A6%95%E0%A7%87+%E0%A6%B8%E0%A6%BF%E0%A6%A8%E0%A7%87%E0%A6%AE%E0%A6%BE+%E0%A6%A6%E0%A7%87%E0%A6%96%E0%A6%A4%E0%A7%87+%E0%A6%AF%E0%A6%BE%E0%A6%AC%E0%A7%87&target_text=%E0%A6%86%E0%A6%9C%E0%A6%95%E0%A7%87+%E0%A6%A4%E0%A7%81%E0%A6%AE%E0%A6%BF+%E0%A6%B8%E0%A6%BF%E0%A6%A8%E0%A7%87%E0%A6%AE%E0%A6%BE+%E0%A6%A6%E0%A7%87%E0%A6%96%E0%A6%A4%E0%A7%87+%E0%A6%AF%E0%A6%BE%E0%A6%AC%E0%A7%87+%E0%A6%95%E0%A6%BF%3F"
    headers = {"Content-Type": "application/json"}

    payload = {"source_text": source_text, "target_text": target_text}

    response = requests.get(url, headers=headers, json=payload, verify=False)
    response.raise_for_status()  # Check for HTTP request errors
    result = response.json()
    answer = result.get('paraphrase', False)
    return answer

In [ ]:
source_text= "তুমি কি আজকে সিনেমা দেখতে যাবে"
target_text = "আজকে তুমি সিনেমা দেখতে যাবে কি?"

In [ ]:
paraphrase(source_text, target_text)

True

In [ ]:
from datasets import load_dataset
dataset = load_dataset("csebuetnlp/BanglaParaphrase", trust_remote_code=True, split="test")

README.md:   0%|          | 0.00/5.49k [00:00<?, ?B/s]

BanglaParaphrase.py:   0%|          | 0.00/2.53k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/76.1M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/4.22M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/419967 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23332 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23331 [00:00<?, ? examples/s]

In [ ]:
from tqdm.auto import tqdm

correct = 0
total = 0

anomalies = []
num_samples = 1000

subset = dataset.select(range(num_samples))

for sample in tqdm(subset):
    source_text = sample['source']
    target_text = sample['target']


    try:
        # Predict paraphrase using the API
        pred_text = paraphrase(source_text, target_text)
        if pred_text == True:
            correct += 1
        total += 1
    except requests.exceptions.HTTPError:
        # Handle cases where the API fails
        anomalies.append({
            "source_text": source_text,
            "target_text": target_text,
        })

# Calculate and print accuracy
accuracy = correct / total if total > 0 else 0
print(f"Accuracy: {accuracy:.2%}")

# Save anomalies to a CSV file
df = pd.DataFrame.from_records(anomalies)
df.to_csv("anomalies1.csv", index=False)

  0%|          | 0/1000 [00:00<?, ?it/s]

Accuracy: 100.00%
